# Doki Doki Literature Club file extraction

This is a pile of one-off functions and commands to extract the contents of the compressed and obfuscated archives in DDLC.

Note that this notebook targets Python 3. I extracted a few Ren'Py functions and modified them to suit my needs, not the least of which included modifications to correctly handle py3 strings (Ren'Py relies on py2 byte strings). [Original Ren'Py source is here](https://github.com/renpy/renpy).

In [1]:
import os

os.chdir(os.path.expanduser('~/Desktop/'))

game_dir = os.environ.get('GAME_DIR', os.path.expanduser('~/Library/Application Support/Steam/steamapps/common/Doki Doki Literature Club/game/'))
output_dir = os.environ.get('OUTPUT_DIR', 'output')

rpa_paths = []
for filename in os.listdir(game_dir):
    if filename.endswith('.rpa'):
        rpa_paths.append(os.path.join(game_dir, filename))

In [2]:
# based on renpy's loader.index_archives function
# see: https://github.com/renpy/renpy/blob/master/renpy/loader.py

from codecs import decode
from pickle import loads

def index_archive(rpa_path):
    prefix = os.path.basename(rpa_path)[:-4]
    with open(rpa_path, 'rb') as f:
        l = f.readline()
        if l.startswith(b'RPA-3.0 '):
            offset = int(l[8:24], 16)
            key = int(l[25:33], 16)
            f.seek(offset)
            index = loads(decode(f.read(), encoding='zlib'))
            for k in index.keys():
                if len(index[k][0]) == 2:
                    index[k] = [ (offset ^ key, dlen ^ key) for offset, dlen in index[k] ]
                else:
                    index[k] = [ (offset ^ key, dlen ^ key, start) for offset, dlen, start in index[k] ]

            return (prefix, index)
        else:
            raise Exception('bad format: {}'.format(l))

In [3]:
# based on renpy's SubFile class
# see: https://github.com/renpy/renpy/blob/master/renpy/loader.py

class SubFile(object):

    def __init__(self, fn, base, length, start):
        self.fn = fn

        self.f = None

        self.base = base
        self.offset = 0
        self.length = length
        
        if type(start) is str:
            start = start.encode('utf-8')
        
        self.start = start

        if not self.start:
            self.name = fn
        else:
            self.name = None

    def open(self):
        self.f = open(self.fn, 'rb')
        self.f.seek(self.base)

    def __enter__(self):
        return self

    def __exit__(self, _type, value, tb):
        self.close()
        return False

    def read(self, length=None):
        if self.f is None:
            self.open()

        maxlength = self.length - self.offset

        if length is not None:
            length = min(length, maxlength)
        else:
            length = maxlength

        rv1 = self.start[self.offset:self.offset + length]
        length -= len(rv1)
        self.offset += len(rv1)

        if length:
            rv2 = self.f.read(length)
            self.offset += len(rv2)
        else:
            rv2 = b''

        return (rv1 + rv2)

    def close(self):
        if self.f is not None:
            self.f.close()
            self.f = None

In [4]:
# based on renpy's loader.load_code function
# see: https://github.com/renpy/renpy/blob/master/renpy/loader.py

def build_file(prefix, loc_tuple):
    filename = prefix + '.rpa'
    rpa_path = os.path.join(game_dir, filename)
    if len(loc_tuple) == 2:
        offset, dlen = loc_tuple
        start = b''
    else:
        offset, dlen, start = loc_tuple
    return SubFile(rpa_path, offset, dlen, start)

In [5]:
# finally, it's time to write some files to disk

import pathlib
import shutil

archives = [index_archive(rpa_path) for rpa_path in rpa_paths]

for prefix, index in archives:
    for outpath, loc_tuple_list in index.items():
        write_path = os.path.join(output_dir, outpath)
        out_dir = os.path.dirname(write_path)
        pathlib.Path(out_dir).mkdir(parents=True, exist_ok=True)
        sub_file = build_file(prefix, loc_tuple_list[0])
        with open(write_path, 'wb') as outf, sub_file as inf:
            print(write_path)
            shutil.copyfileobj(inf, outf)

output/bgm/heartbeat.ogg
output/bgm/10.ogg
output/sfx/slap.ogg
output/bgm/5_ghost.ogg
output/sfx/glitch1.ogg
output/bgm/m1.ogg
output/bgm/monika-end.ogg
output/sfx/eyes.ogg
output/sfx/mscare.ogg
output/bgm/g2.ogg
output/bgm/1.ogg
output/bgm/6.ogg
output/bgm/6s.ogg
output/bgm/4.ogg
output/bgm/5.ogg
output/sfx/stab.ogg
output/bgm/g1.ogg
output/bgm/5_sayori.ogg
output/gui/font/VerilySerifMono.otf
output/sfx/interference.ogg
output/sfx/crack.ogg
output/bgm/2.ogg
output/gui/sfx/baa.ogg
output/bgm/7g.ogg
output/bgm/s_kill_early.ogg
output/sfx/s_kill_glitch1.ogg
output/sfx/yuri-kill.ogg
output/gui/sfx/select.ogg
output/sfx/monikapound.ogg
output/sfx/closet-close.ogg
output/gui/sfx/select_glitch.ogg
output/bgm/7.ogg
output/bgm/4g.ogg
output/sfx/fall.ogg
output/sfx/closet-open.ogg
output/bgm/3.ogg
output/bgm/9g.ogg
output/bgm/3g.ogg
output/sfx/pageflip.ogg
output/bgm/d.ogg
output/sfx/smack.ogg
output/bgm/6g.ogg
output/sfx/gnid.ogg
output/bgm/ghostmenu.ogg
output/sfx/glitch3.ogg
output/bgm/9.ogg

output/images/yuri/a2.png
output/images/poem_special/poem_end.png
output/images/bg/poem.jpg
output/images/cg/y_cg2_dust4.png
output/images/cg/monika/monika_glitch4.png
output/gui/textbox.png
output/images/natsuki/2bti.png
output/images/yuri/dragon2.png
output/gui/poemgame/y_sticker_2g.png
output/images/sayori/r.png
output/gui/skip.png
output/images/cg/y_cg2_base.png
output/images/sayori/old/2.png
output/images/natsuki/2btc.png
output/gui/menu_bg_m.png
output/gui/poemgame/n_sticker_2.png
output/images/yuri/glitch2.png
output/images/cg/monika/monika_glitch1.png
output/images/sayori/c.png
output/gui/phone/overlay/game_menu.png
output/images/monika/old/5.png
output/images/natsuki/2tb.png
output/images/monika/old2/2r.png
output/images/cg/s_kill_bg.png
output/images/natsuki/old2/1t.png
output/images/cg/y_kill/2c.png
output/images/natsuki/v.png
output/images/bg/glitch-red.png
output/gui/button/idle_background.png
output/images/bg/poem-glitch2.png
output/images/cg/n_cg3_exp1.png
output/images/